In [1]:
pip install galois

Note: you may need to restart the kernel to use updated packages.


In [3]:
import galois
import numpy as np
import time

In [15]:
#definisco un campo di Galois come nell'esempio 
field = galois.GF(8)

In [16]:
#definisco una matrice k x (n-k) dove n-k = 4
k = 3 #le righe
n_k = 4 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 

In [17]:
#ho verificato che funziona pure con i random
#A = field.Random((k, n_k))
#print(A)

#questa è con l'esempio di 4.3
A = field([[1,2,4,7],
           [0,2,3,1],
           [2,3,2,0]])

In [18]:
matrice_bin1 = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [0, 1, 0]
])

matrice_bin1 = field(matrice_bin1)

matrice_bin2 = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 1, 0, 0]
])

matrice_bin2 = field(matrice_bin2)

In [19]:
A_primo = field(matrice_bin1 @ A @ matrice_bin2)
print(A_primo)

[[2 0 3 2]
 [1 7 2 4]
 [0 1 2 3]]


In [57]:
A_primo = field([[2,2,0,3],
           [2,0,2,3],
           [0,3,1,2]])

In [20]:
#definito funzione ordinamento lessicografico riga (ordino ogni riga e definisco ordine degli indici) e gestiti casi di 
#conflitto. L'argomento colonne è quello che ci indica in che ordine verificare i conflitti 
#ex: colonne=[0,1,2] --> ordino per colonna 0, se c'è conflitto tra due vedo la colonna 1 e cosi via

#faccio ordinamento crescente dei valori all'interno della riga e torno matrice con righe ordinate (ordinato).
#da questa matrice faccio l'ordinamento elemento a elemento posizionale corrispondente e definisco quindi gli indici 
#riordinati. ex: [1 0 2] --> prima la riga ORIGINALE 1 poi riga ORIGINALE 0 e così via. Per originale intendo "matrice"

#questa è la funzione che gli fornisco la matrice ordinata all'interno delle righe e lei pme la ordina per righe
def ordina_matrice(matrice):
    print(matrice)
    #righe matrice
    n = matrice.shape[0]  
    #contatore confronti
    confronti = 0  
    #array indici originali
    indici = list(np.arange(n))  
    print("Matrice ordinata:\n", matrice) #QUESTO AI FINI DI VELOCITA LO POSSO TOGLIERE

    #bubble sort per conteggio confronti
    for i in range(n):
        for j in range(i + 1, n):
            print(f"Confronto tra riga {i} e riga {j}")
            #confronto per primo el riga poi secondo e cosi via
            for k in range(matrice.shape[1]):  
                confronti += 1
                #sono ordinate tra loro, vado avanti
                if matrice[indici[i], k] < matrice[indici[j], k]: 
                    print(f"Riga {i} ({matrice[indici[i]]}) è minore di Riga {j} ({matrice[indici[j]]})")
                    break
                #devo mettere la riga sopra quella con cui è confrontata e andare avanti
                elif matrice[indici[i], k] > matrice[indici[j], k]:
                    print(f"Riga {i} ({matrice[indici[i]]}) è maggiore di Riga {j} ({matrice[indici[j]]})")
                    #sposto indice sopra
                    indici[i], indici[j] = indici[j], indici[i]
                    print("Indici dopo lo spostamento:", indici)
                    break 
                if k == (n_k - 1):
                    print("Algoritmo ha almeno due righe uguali --> fallisco")
                    return [], confronti
    print("Indici delle righe ordinate:", indici)
    print("Numero totale di confronti:", confronti)
    
    return indici, confronti



def ordina_per_riga(matrice):
    ordinato = []
    for riga in matrice:
        ordinato.append(np.sort(riga))
        
    ordinato = np.array(ordinato)
    
    indici_ordinati, num_confronti = ordina_matrice(ordinato)
    if len(indici_ordinati) == 0:
        print("Fasullo")
        return [[]]
    print("Matrice ordinata per righe: ",matrice[indici_ordinati])
    return matrice[indici_ordinati]

#ordino poi per il primo elemento della colonna e in caso di conflitti li ho gestiti. 
def ordina_per_colonna(matrice, righe):
    if len(matrice) == 0:
        return [[]]
    indici_ordinamento = np.lexsort([matrice[rig, :] for rig in righe[::-1]])
    return matrice[:, indici_ordinamento]

#qui faccio il test
def test_uguaglianza(matrice1, matrice2):
    colonne = range(n_k)
    righe = range(k)
    print("Matrice1: ", matrice1)

    ordinata1 = ordina_per_colonna(ordina_per_riga(matrice1), righe)
    print("\nOrdinata 1: ",ordinata1)

    print("Matrice2: ", matrice2)
    ordinata2 = ordina_per_colonna(ordina_per_riga(matrice2), righe)
    print("\nOrdinata2: ",ordinata2)     

    if np.array_equal(ordinata1, ordinata2):
        print("Le due matrici hanno la stessa CF.")
        return True
    else:
        print("Le due matrici hanno CF diversa")
        return False   



In [21]:
test_uguaglianza(A, A_primo)

Matrice1:  [[1 2 4 7]
 [0 2 3 1]
 [2 3 2 0]]
[[1 2 4 7]
 [0 1 2 3]
 [0 2 2 3]]
Matrice ordinata:
 [[1 2 4 7]
 [0 1 2 3]
 [0 2 2 3]]
Confronto tra riga 0 e riga 1
Riga 0 ([1 2 4 7]) è maggiore di Riga 1 ([0 1 2 3])
Indici dopo lo spostamento: [np.int64(1), np.int64(0), np.int64(2)]
Confronto tra riga 0 e riga 2
Riga 0 ([0 1 2 3]) è minore di Riga 2 ([0 2 2 3])
Confronto tra riga 1 e riga 2
Riga 1 ([1 2 4 7]) è maggiore di Riga 2 ([0 2 2 3])
Indici dopo lo spostamento: [np.int64(1), np.int64(2), np.int64(0)]
Indici delle righe ordinate: [np.int64(1), np.int64(2), np.int64(0)]
Numero totale di confronti: 4
Matrice ordinata per righe:  [[0 2 3 1]
 [2 3 2 0]
 [1 2 4 7]]

Ordinata 1:  [[0 1 2 3]
 [2 0 3 2]
 [1 7 2 4]]
Matrice2:  [[2 0 3 2]
 [1 7 2 4]
 [0 1 2 3]]
[[0 2 2 3]
 [1 2 4 7]
 [0 1 2 3]]
Matrice ordinata:
 [[0 2 2 3]
 [1 2 4 7]
 [0 1 2 3]]
Confronto tra riga 0 e riga 1
Riga 0 ([0 2 2 3]) è minore di Riga 1 ([1 2 4 7])
Confronto tra riga 0 e riga 2
Riga 0 ([0 2 2 3]) è maggiore di Rig

True

In [12]:
#FACCIO METODO DI MONTECARLO
#numero iterazioni Monte Carlo
num_iterazioni = 1500  

#QUESTA POTREBBE NON SERVIRE
# Funzione per generare matrici binarie casuali
"""def genera_matrice_binaria(dimensioni, campo):
    matrice_binaria = np.zeros(dimensioni, dtype = int)
    perm = np.random.permutation(dimensioni[0])
    for i in range(dimensioni[0]):
        matrice_binaria[i, perm[i]] = 1
    return campo(matrice_binaria)"""

#fa il procedimento di ordinamento tra le righe (che pero sono già ordinate al loro interno)
def ordina_matrice(matrice):
    print("Entro in ordina matrice")
    #righe matrice
    n = matrice.shape[0]  
    #contatore confronti
    confronti = 0  
    confronti1 = 0
    #array indici originali
    indici = np.arange(n)  
    indici1 = np.arange(n)
    #print(matrice)

    esci = False
    #bubble sort per conteggio confronti
    for i in range(n):
        if esci:
            break
        for j in range(i + 1, n):
            if esci:
                break
            #print(f"Confronto tra riga {i} e riga {j}")
            #confronto per primo el riga poi secondo e cosi via
            for k in range(matrice.shape[1]):  
                confronti += 1
                #sono ordinate tra loro, vado avanti
                if matrice[indici[i], k] < matrice[indici[j], k]: 
                    #print(f"Riga {i} ({matrice[indici[i]]}) è minore di Riga {j} ({matrice[indici[j]]})")
                    break
                #devo mettere la riga sopra quella con cui è confrontata e andare avanti
                elif matrice[indici[i], k] > matrice[indici[j], k]:
                    #print(f"Riga {i} ({matrice[indici[i]]}) è maggiore di Riga {j} ({matrice[indici[j]]})")
                    #sposto indice sopra
                    indici[i], indici[j] = indici[j], indici[i]
                    #print("Indici dopo lo spostamento:", indici)
                    break 
                if k == (n_k - 1):
                    print("Algoritmo ha almeno due righe uguali --> fallisco")
                    return False, [], confronti, confronti1

    esci = False
    for i in range(n):
        if esci:
            break
        for j in range(i + 1, n):
            if esci:
                break
            #print(f"Confronto tra riga {i} e riga {j}")
            #confronto per primo el riga poi secondo e cosi via
            for k in range(10):  
                confronti1 += 1
                #sono ordinate tra loro, vado avanti
                if matrice[indici1[i], k] < matrice[indici1[j], k]: 
                    #print(f"Riga {i} ({matrice[indici[i]]}) è minore di Riga {j} ({matrice[indici[j]]})")
                    break
                #devo mettere la riga sopra quella con cui è confrontata e andare avanti
                elif matrice[indici1[i], k] > matrice[indici1[j], k]:
                    #print(f"Riga {i} ({matrice[indici[i]]}) è maggiore di Riga {j} ({matrice[indici[j]]})")
                    #sposto indice sopra
                    indici1[i], indici1[j] = indici1[j], indici1[i]
                    #print("Indici dopo lo spostamento:", indici)
                    break 
                if k == (10 - 1):
                    print("Algoritmo ha almeno due righe (CORTE) uguali --> fallisco")
                    return False, [], confronti, confronti1
                
                #se uguali continua a confrontare la prossima colonna

    #print("Matrice ordinata:\n", matrice) #QUESTO AI FINI DI VELOCITA LO POSSO TOGLIERE
    #print("Indici delle righe ordinate:", indici)
    #print("Numero totale di confronti:", confronti)
    #POSSO  EVITARE IL RETURN DELLA MATRICE
    #print(matrice[indici], matrice[indici1])
    #print(indici, indici1)
    #print(np.array_equal(matrice[indici], matrice[indici1]))
    #print(matrice[indici])
    #print(matrice[indici1])
    print("Confronti = ", confronti, "Confronti 1 = ", confronti1)
    if np.array_equal(matrice[indici], matrice[indici1]):
        print("If positivo")
        #print("Indici:", indici)
        return True, indici, confronti, confronti1
    else:
        print("If negativo")
        return False, [], confronti, confronti1
# Funzione per ordinare le righe con contatori di ordinamenti e swap (TOGLI COLONNE). In num_confronti metto confronti in 
# sufficienti metto confronti1
def ordina_per_riga_con_contatori(matrice, num_swap_righe, num_confronti, conta_sufficienti):
    #vediamo se 5 colonne sono sufficienti per il numero di confronti nelle righe
    # Mantieni traccia della posizione originale delle righe
    print("Funzione ordina_per_riga")
    posizioni_originali = np.arange(matrice.shape[0])
    
    # Ordina gli elementi all'interno di ogni riga
    ordinato = []
    for riga in matrice:
        ordinato.append(np.sort(riga))
    
    ordinato = np.array(ordinato)
    ris_confronto, indici_ordinamento, num_confronti,conta_sufficienti = ordina_matrice(ordinato)

    if ris_confronto:
        #print(f"Qui ho indici ordinamento: {indici_ordinamento}")
        # Trova gli indici di ordinamento usando np.lexsort sulle colonne
        #indici_ordinamento = np.lexsort([ordinato[:, col] for col in colonne[::-1]])
        # Conta gli swap solo quando la posizione delle righe cambia
        for i, indice in enumerate(indici_ordinamento):
            if indice != posizioni_originali[i]:
                num_swap_righe += 1
                #num confronti qui non dovrebbe servire
        return matrice[indici_ordinamento], num_swap_righe, num_confronti, conta_sufficienti
    else:
        return [], num_swap_righe, num_confronti, conta_sufficienti


#POTREI DOVER IMPLEMENTARE UNA COSA SIMILE A SOPRA PER IL COUNT DEI CONFRONTI (LEXSORT NON CE L'HA)
#ordinare le colonne con contatori di ordinamenti e swap
def ordina_per_colonna_con_contatori(matrice, righe, num_swap_colonne):
    print("Funzione ordina per colonna")
    # Mantieni traccia della posizione originale delle colonne
    posizioni_originali = np.arange(matrice.shape[1])
    
    # Trova gli indici di ordinamento usando np.lexsort sulle righe
    indici_ordinamento = np.lexsort([matrice[rig, :] for rig in righe[::-1]])
        
    # Conta gli swap solo quando la posizione delle colonne cambia
    for i, indice in enumerate(indici_ordinamento):
        if indice != posizioni_originali[i]:
            num_swap_colonne += 1
    
    # Riordina la matrice in base agli indici trovati
    matrice_ordinata = matrice[:, indici_ordinamento]
    # Faccio num_swap_colonne/2 poichè contando il numero di diversi ne conta ovviamente il doppio. 
    # Ex: (0, 1, 2, 3) potrebbe diventare (0, 1, 3, 2) il contatore sarebbe a due, ma io ho fatto uno swap
    return matrice_ordinata, num_swap_colonne

#IL NOME DELLA FUNZIONE È FORVIANTE E POSSO ELIMINARE MATRICE 2 PERCHÈ NON C'È NESSUN CONFRONTO
def test_uguaglianza_con_contatori(matrice1, num_swap_righe, num_swap_colonne, num_confronti, conta_sufficienti, durata):
    print("Funzione test uguaglianza")
    start_time = time.perf_counter()
    #intervallo per le colonne
    #colonne = range(n_k)  
    #intervallo corretto per le righe
    righe = range(k)      
    # Ordinamento matrice1
    #esito, matrice_confronti, indici, num_confronti = ordina_matrice(matrice1)
    #print(f"Esito: {esito}, Matrice confronti: {matrice_confronti}, Indici: {indici}, Num confronti: {num_confronti}")
    #if not esito:
        #print("Entro nell'if di esito")
        #return False, num_swap_righe, num_swap_colonne, num_confronti, conta_sufficienti
    print("Faccio ordina per riga con contatori")
    ordinata1, num_swap_righe, num_confronti, conta_sufficienti = ordina_per_riga_con_contatori(matrice1, 
                                                                num_swap_righe, num_confronti, conta_sufficienti)
    print("Ordino le colonne")
    if len(ordinata1) == 0:
        end_time = time.perf_counter()
        durata = (end_time - start_time)*1000
        return False, num_swap_righe, num_swap_colonne, num_confronti, conta_sufficienti, durata
    

    cf1, num_swap_colonne = ordina_per_colonna_con_contatori(ordinata1, righe, num_swap_colonne)
    # Ordinamento matrice2
    #ordinata2, num_swap_righe = ordina_per_riga_con_contatori(matrice2, colonne, num_swap_righe, num_confronti)
    #cf2, num_swap_colonne = ordina_per_colonna_con_contatori(ordinata2, righe, num_swap_colonne)
    end_time = time.perf_counter()
    durata = (end_time - start_time)*1000
    print("devo fare solo il return")
    # Confronto
    # Vedo prima se prese le due matrici a meno di due righe sono uguali E poi se effettivamente sono uguali allora ritorno True
    #sono un coglione tanto l'uguaglianza è dimostrata, DEVO EVITARE DI CONSIDERARE LE DUE RIGHE SULL'RODINAMENTO DELLE RIGHE
    #TUTTA L'ANALISI DI MONTECARLO NON LA DEVO FARE QUA, MA LA DEVO FARE SULLA FUNZIONE DEL CONFRONTA ELEMENTO A ELEMENTO PER 
    #EVITARE DI VEDERLI TUTTI
    # QUINDI UNA VOLTA CHE HO LA MATRICE A DEVO FARE IL PROCESSO DI ORDINAMENTO DELLE RIGHE O SENZA CONSIDERARE GLI ULTIMI DUE 
    # ELEMENTI DELLA COLONNA (PER ESEMPIO) OPPURE CONSIDERANDOLI E POI NON ANDANDOLI A CONFRONTARE AD UNO A UNO CON QUELLI 
    # DELLA RIGA IN CONFLITTO
    #la forma canonica potrebbe non interessarmi (cf1)
    return True, num_swap_righe, num_swap_colonne, num_confronti, conta_sufficienti, durata

# Funzione per eseguire la simulazione di Monte Carlo
def simulazione_montecarlo(num_iterazioni):
    #contatore simulazioni positive
        #simulazione accettabile
    simulazione_accettabile = 0
    #contatore swap righe
    num_swap_righe = 0  
    #contatore swap colonne
    num_swap_colonne = 0  
    #contatore confronti righe
    num_confronti_accumulatore = 0
    conta_sufficienti_accumulatore = 0
    #inizializzo la durata
    tempo_esecuzione = 0.0
    durata = 0
    num_confronti = 0
    conta_sufficienti = 0

    
    for z in range(num_iterazioni):
        print(f"ITERAZIONE {z}")
        #field.order mi fa generare i numeri da 0 a 7 (field.order = 8)
        A = field(np.random.randint(0, field.order, size=(k, n_k)))
        #matrice_bin1 = genera_matrice_binaria((k, k), field)
        #matrice_bin2 = genera_matrice_binaria((n_k, n_k), field)

        #A_primo = field(matrice_bin1 @ A @ matrice_bin2)
        esito, num_swap_righe, num_swap_colonne, num_confronti, conta_sufficienti, durata = test_uguaglianza_con_contatori(A, 
                                                            num_swap_righe, num_swap_colonne, num_confronti, conta_sufficienti, durata)
        num_confronti_accumulatore += num_confronti
        conta_sufficienti_accumulatore += conta_sufficienti
        print("Num confronti cumulativo: ", num_confronti_accumulatore, "Conta sufficienti cumulativo: ", conta_sufficienti_accumulatore)

        #calcolo il tempo di ogni esecuzione, pure quelle negative
        tempo_esecuzione += durata
        if esito:
            simulazione_accettabile += 1
        #if uguaglianza:
            #simulazioni_positive += 1
        #    simulazione_accettabile += 1
                
    return tempo_esecuzione/num_iterazioni, num_swap_righe/num_iterazioni, num_swap_colonne/num_iterazioni, num_confronti_accumulatore/num_iterazioni, conta_sufficienti_accumulatore/num_iterazioni, (simulazione_accettabile/num_iterazioni)*100 
    


In [13]:
# Output statistiche
# Esegui la simulazione
tempo_es_medio, num_medio_sw_ri, num_medio_sw_co, num_medio_confronti, num_medio_conf_sufficienti, accettati_perc = simulazione_montecarlo(num_iterazioni)



ITERAZIONE 0
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice
Confronti =  20629 Confronti 1 =  20629
If positivo
Ordino le colonne
Funzione ordina per colonna
devo fare solo il return
Num confronti cumulativo:  20629 Conta sufficienti cumulativo:  20629
ITERAZIONE 1
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice
Confronti =  21064 Confronti 1 =  21064
If positivo
Ordino le colonne
Funzione ordina per colonna
devo fare solo il return
Num confronti cumulativo:  41693 Conta sufficienti cumulativo:  41693
ITERAZIONE 2
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice
Confronti =  19734 Confronti 1 =  19734
If positivo
Ordino le colonne
Funzione ordina per colonna
devo fare solo il return
Num confronti cumulativo:  61427 Conta sufficienti cumulativo:  61427
ITERAZIONE 3
Funzione test uguaglianza
Faccio 

In [14]:
print()
print(f"Tempo esecuzione medio: {tempo_es_medio} ms")
print(f"Numero medio di swap righe: {num_medio_sw_ri}")
print(f"Numero medio di swap colonne: {num_medio_sw_co}")
print(f"Numero medio di confronti tra le righe in una matrice: {num_medio_confronti}")
print(f"Numero medio di confronti con meno colonne: {num_medio_conf_sufficienti}")
print(f"Percentuale di accettati (non controllo tutte le righe, mi fermo a x colonne prima): {accettati_perc}%")


Tempo esecuzione medio: 83.16930453000143 ms
Numero medio di swap righe: 110.35466666666666
Numero medio di swap colonne: 110.30866666666667
Numero medio di confronti tra le righe in una matrice: 20525.912
Numero medio di confronti con meno colonne: 18848.493333333332
Percentuale di accettati (non controllo tutte le righe, mi fermo a x colonne prima): 88.26666666666667%
